*Temporary Style Settings here*
<style>
li {
    list-style: disc;
    margin-left: 2em;
}
li p {
    list-style: disk;
    line-height: normal;
    margin-bottom: 0;
}
table p {
    list-style: disk;
    line-height: normal;
    margin: 0 2em
    ;
    text-align: left;
}
</style>

# Putting it in **Context**

Every time a sequence is applied to a section a _context_ attribute is created.
The _context_ attribute is a dictionary object containing information about the 
section and the sequence which can be accessed by _Section_break_, _process_, 
and _assembly_ functions.  Custom items can also be included in _context_, 
allowing information to be passed from one function to another within the 
section. 

## Standard Items in _Context_

The following table describes the _context_ items that are automatically 
generated by the Section class.

<table>
<thead><th>Item Key</th><th>Value Description</th></thead>
<tr><td>Current Section</td>
<td>The assigned name of the section: <i>self.section_name</i></td></tr>

<tr><td>Skipped Lines</td>
<td>A list of items in the source occurring before the first item of the section</td></tr>

<tr><td>Status</td>
<td>The current section scanning status one of:<ul>
    <li>Not Started</li>
    <li>At section start</li>
    <li>Scan In Progress</li>
    <li>Break Triggered</li>
    <li>Scan Complete</li>
    <li>End of Source</li>
    </ul></td></tr>
<tr><td>Break</td>
<td>The name assigned to the SectionBreak that most recently triggered. 
<ul>
    <li>Before the section start this is not defined.</li>
    <li>While the section scan is in progress, 
    this is the name of the StartSection.</li> 
    <li>After the section ends, this is the name of the the EndSection.</li>
    </ul>
    </td></tr>
<tr><td>
Event</td>
<td>The result from the SectionBreak that most recently triggered; the 
<i>TriggerEvent.test_value</i>.  
For more information see the Triggers documentation.</td></tr>
</table>


## Accessing Default Context Values

### Accessing Context With a Processor Function
* Add a second argument to the processor function.  
* It will receive the context dictionary

In [7]:
# pprint is used to produce nicely formatted output.
from pprint import pprint

# Import Section and SectionBreak
from sections import Section, SectionBreak

# Sample text source
source = [
    'Item before the section starts',
    'Section line A1',
    'Item after section ends'
    ]

# Process function that prints the section's context
def process_show_context(input_item, process_context):
    '''display the section's context and return the section_item.
    '''
    print('\nContext in Processor:')
    pprint(process_context)
    return input_item


# Define starting and ending section boundaries
# Start with the first line containing the text 'Section'
start_boundary = SectionBreak('Section', name='Start Section')
# Ends before the first line containing the text 'ends'
end_boundary = SectionBreak('ends', name='End Section')

# Define the section `context_content` 
context_content = Section(start_section=start_boundary, 
                                   end_section=end_boundary,
                                   processor=process_show_context,
                                   section_name='Process Context Content')


# Read the `source` text using `context_content` 
output = context_content.read(source)


Context in Processor:
{'Break': 'Start Section',
 'Current Section': 'Process Context Content',
 'Event': 'Section',
 'Skipped Lines': ['Item before the section starts'],
 'Status': 'Scan In Progress'}


**Context Content:**

Break: Start Section
    The name assigned to the SectionBreak that most recently triggered.
    Since processing happens inside the section, the most recent boundary is the 
    section start.

Current Section: Process Context Content
    The assigned name of the section.  This name was given when the section 
    object was created. 
    `context_content = Section(` $\cdots$ `section_name='Process Context Content')`

Event: Section
    The result from the SectionBreak that most recently triggered.  For 
    string-based section breaks this is the text that was found that triggered 
    the section break.

Skipped Lines: <code>['Item before the section starts']</code>
    A list of items in the source occurring before the first item of the 
    section. In this case there was one line before the beginning of the 
    section: *'Item before the section starts'*

Status: Scan In Progress
    The current section scanning status. Because this is within a processing
     function, it should be *'Scan In Progress'*.

### Accessing Context With an Assembly Function
* Add a second argument to the assembly function.  
* It will receive the context dictionary

In [8]:
# pprint is used to produce nicely formatted output.
from pprint import pprint

# Import Section and SectionBreak
from sections import Section, SectionBreak

# Sample text source
source = [
    'Item before the section starts',
    'Section line A1',
    'Section line A2',
    'Item after section ends'
    ]

# Assembly function that prints the section's context
def assembly_show_context(processed_items, context):
    '''display the section's context and return the item list.
    '''
    section_list = [processed_item for processed_item in processed_items]
    print('\nContext in Assembler:')
    pprint(context)
    print('\n')
    return section_list

# Define starting and ending section boundaries
# Start with the first line containing the text 'Section'
start_boundary = SectionBreak('Section', name='Start Section')
# Ends before the first line containing the text 'ends'
end_boundary = SectionBreak('ends', name='End Section')

# Define the section `context_content` 
context_content = Section(start_section=start_boundary, 
                                   end_section=end_boundary,
                                   aggregate=assembly_show_context,
                                   section_name='Assembly Context Content')


# Read the `source` text using `context_content` 
output = context_content.read(source)


Context in Assembler:
{'Break': 'End Section',
 'Current Section': 'Assembly Context Content',
 'Event': 'ends',
 'Skipped Lines': ['Item before the section starts'],
 'Status': 'Break Triggered'}




**Context Content:**

Break: End Section
    The name assigned to the SectionBreak that most recently triggered.
    Since assembly happens after the section is completed, the most recent 
    boundary is the section end.

Current Section: Assembly Context Content
    The assigned name of the section.  This name was given when the section 
    object was created. 
    `context_content = Section(` $\cdots$ `section_name='Assembly Context Content')`

Event: ends
    The result from the SectionBreak that most recently triggered.  For 
    string-based section breaks this is the text that was found that triggered 
    the section break.

Skipped Lines: <code>['Item before the section starts']</code>
    A list of items in the source occurring before the first item of the 
    section. In this case there was one line before the beginning of the 
    section: *'Item before the section starts'*

Status: Break Triggered
    The current section scanning status. Because this is within a assembly
     function, it should be either *'Break Triggered'* or *'End of Source'*.

### Passing an External Value to a Processing or Assembly Function


In [10]:
# pprint is used to produce nicely formatted output.
from pprint import pprint

# Import Section and SectionBreak
from sections import Section, SectionBreak

# Sample text source
source = [
    'Item before the section starts',
    'Section line A1',
    'Section line A2',
    'Item after section ends'
    ]

# Process function that prints the section's context
def process_show_context(input_item, context):
    '''display the section's context and return the section_item.
    '''
    print('\nContext in Processor:')
    pprint(context)
    return input_item


# Assembly function that prints the section's context
def assembly_show_context(processed_items, context):
    '''display the section's context and return the item list.
    '''
    section_list = [processed_item for processed_item in processed_items]
    print('\nContext in Assembler:')
    pprint(context)
    print('\n')
    return section_list

# Define starting and ending section boundaries
# Start with the first line containing the text 'Section'
start_boundary = SectionBreak('Section', name='Start Section')
# Ends before the first line containing the text 'ends'
end_boundary = SectionBreak('ends', name='End Section')

# Define the section `context_content` 
context_content = Section(start_section=start_boundary, 
                                   end_section=end_boundary,
                                   processor=process_show_context,
                                   aggregate=assembly_show_context,
                                   section_name='Context Content')


# Read the `source` text using `context_content` 
pprint(context_content.read(source, dummy_variable='Hello'))

TypeError: read() got an unexpected keyword argument 'dummy_variable'

In [ ]:
# pprint is used to produce nicely formatted output.
from pprint import pprint

# Import Section and SectionBreak
from sections import Section

# This is the demo input we will use.
price_list = [
    'MODEL NAME, CPU, RAM, PRICE',
    'THINKCENTRE X1, Core i5/6200, 8, $260',
    'DELL OPTIPLEX 790, Core i5/2500, 4, $20'
    ]

# Process function that prints the section's context
def process_show_context(input_item, context):
    '''display the section's context and return the section_item.
    '''
    print('\nContext in Processor:')
    pprint(context)
    print('\n')
    return input_item


# Assembly function that prints the section's context
def assembly_show_context(processed_items, context):
    '''display the section's context and return the item list.
    '''
    print('\nContext in Assembler:')
    pprint(context)
    print('\n\n')
    return [processed_item for processed_item in processed_items]


# Define the section `price_all_in_one_section` 
# Define the starting boundary: After the line that starts with 'MODEL'
# Define the processor: split the text at each ',' remove the '\$' and spaces,
#   convert the first and last columns to a two-item tuple.
# Define the Assembler: Convert the 2-item tuple into a dictionary.  
price_all_in_one_section = Section(start_section=('MODEL', 'START', 'After', 
                                                  'Start Section'),
                                   processor=process_show_context,
                                   aggregate=assembly_show_context)


# Read the `price_list` text using the `price_all_in_one_section` 
pprint(price_all_in_one_section.read(price_list))


Context in Assembler:
{'Break': 'Start Section',
 'Current Section': 'Section',
 'Event': 'MODEL',
 'Skipped Lines': ['MODEL NAME, CPU, RAM, PRICE'],
 'Status': 'At section start'}




Context in Processor:
{'Break': 'Start Section',
 'Current Section': 'Section',
 'Event': 'MODEL',
 'Skipped Lines': ['MODEL NAME, CPU, RAM, PRICE'],
 'Status': 'Scan In Progress'}



Context in Processor:
{'Break': 'Start Section',
 'Current Section': 'Section',
 'Event': 'MODEL',
 'Skipped Lines': ['MODEL NAME, CPU, RAM, PRICE'],
 'Status': 'Scan In Progress'}


['THINKCENTRE X1, Core i5/6200, 8, $260',
 'DELL OPTIPLEX 790, Core i5/2500, 4, $20']


In [ ]:
# pprint is used to produce nicely formatted output.
from pprint import pprint

# Import Section and SectionBreak
from sections import Section

# This is the demo input we will use.
price_list = [
    'MODEL NAME, CPU, RAM, PRICE',
    'THINKCENTRE X1, Core i5/6200, 8, $260',
    'THINKCENTRE M78, AMD A8-6500, 8, $30',
    'THINKCENTRE M53, Celeron, 8, $60',
    'THINKCENTRE M710Q, Intel Pentium, 8, $40',
    'DELL OPTIPLEX 7060, Core i7-8700, 8, $385',
    'DELL OPTIPLEX 790, Core i5/2500, 4, $20'
    ]

# Mini-functions for each processing action
def csv_parse(text):
    '''Split the supplied string at every occurrence of a comma.'''
    return text.split(',')


def drop_space(text_list):
    '''Remove space from the start and end of each substring.'''
    return [txt.strip() for txt in text_list]


# Assembly function
def tuples_to_dict(text_tuples, context):
    '''Convert a sequence of 2-item tuples into a item dictionary 
    with the first tuple element as the key and the second as a float value.
    '''
    # Use a dictionary generator to take each two-element tuple, set the first 
    # as the dictionary key and convert the second into a float value.
    combined_dict = {row[0]: float(row[1]) for row in text_tuples}
    pprint(context)
    return combined_dict


# Define the section `price_all_in_one_section` 
# Define the starting boundary: After the line that starts with 'MODEL'
# Define the processor: split the text at each ',' remove the '\$' and spaces,
#   convert the first and last columns to a two-item tuple.
# Define the Assembler: Convert the 2-item tuple into a dictionary.  
price_all_in_one_section = Section(start_section=('MODEL', 'START', 'After'),
                                   processor=[csv_parse, drop_space],
                                   aggregate=tuples_to_dict)


# Read the `price_list` text using the `price_all_in_one_section` 
pprint(price_all_in_one_section.read(price_list))

{'Break': 'SectionBreak',
 'Current Section': 'Section',
 'Event': 'MODEL',
 'Skipped Lines': ['MODEL NAME, CPU, RAM, PRICE'],
 'Status': 'End of Source'}
{'DELL OPTIPLEX 7060': 385.0,
 'DELL OPTIPLEX 790': 20.0,
 'THINKCENTRE M53': 60.0,
 'THINKCENTRE M710Q': 40.0,
 'THINKCENTRE M78': 30.0,
 'THINKCENTRE X1': 260.0}



- Compare `full_section` and `sub_section` ___Context___ after reading.

Using Context
	3. Using the Context dictionary
	    1. Setting context values when calling Section.read
	    2. 

Built-in Context items
Event
Status

Using context in a processing or aggregate function
Passing a parameter to a processing method
Modifying context
read only context

### Imports

In [ ]:
from typing import List
from pathlib import Path
from pprint import pprint
import re
import sys

import pandas as pd
import xlwings as xw

from buffered_iterator import BufferedIterator
import text_reader as tp
from sections import Rule, RuleSet, SectionBreak, ProcessingMethods, Section

### Display Functions

In [ ]:
#%%writefile display_functions.py

In [ ]:
# %% Function to compare context for two sections.
def compare_context(section1, section2):
    ctx_template = '{key:16s}:\t{item1:16s}\t{item2:16s}'
    context_1 = section1.context
    context_2 = section2.context
    keys_1 = set(context_1.keys())
    keys_2 = set(context_2.keys())
    all_keys = keys_1 | keys_2
    for key in all_keys:
        item1 = context_1.get(key, '')
        item2 = context_2.get(key, '')
        ctx_str = ctx_template.format(key=str(key), item1=str(item1), item2=str(item2))
        print(ctx_str)
        
        
# %% Compare Buffered Iterator contents
def buffered_iterator_compare(iter1, iter2=None, iter3=None, 
                              label1='From Iterator', 
                              label2='To Iterator', label3=''):
    
    def extract_attrs(buf_obj, requested_item, as_list=True):
        if not buf_obj:
            text = ''
        elif as_list:
            text = str(list(buf_obj.__getattribute__(requested_item)))
        else:
            text = str(buf_obj.__getattribute__(requested_item))
        return text
        
    def extract_attr_text(requested_item, iter1, iter2=None, iter3=None, 
                        as_list=True):    
        attr_text = {
            1: extract_attrs(iter1, requested_item, as_list),
            2: extract_attrs(iter2, requested_item, as_list),
            3: extract_attrs(iter3, requested_item, as_list),
        }
        return attr_text


    row_template = ''.join([
        '\t{Label:<20s}',
        '{first_iter_item:<35s}',
        '{second_iter_item:<35s}',
        '{third_iter_item:<35s}\n'
        ])   
    attr_group = {
        'Previous Items': ('previous_items', True),
        'Future Items': ('future_items', True),
        'Item Count': ('item_count', False),
        'Step Back': ('_step_back', False),
        'Buffer Size': ('buffer_size', False)
        }

    row_list = [
        row_template.format(
            Label='',
            first_iter_item=label1, 
            second_iter_item=label2, 
            third_iter_item=label3)
                ]

    for label, attr_s in attr_group.items():
        requested_item, as_list = attr_s
        text_group = extract_attr_text(requested_item, iter1, iter2, iter3, as_list)
        text_line = row_template.format(Label=label, 
                        first_iter_item=text_group[1],
                        second_iter_item=text_group[2],
                        third_iter_item=text_group[3])
        row_list.append(text_line)
    
    iterator_compare_str = ''.join(row_list)
    
    return iterator_compare_str

In [ ]:
GENERIC_TEST_TEXT = [
    'Text to be ignored',
    'StartSection Name: A',
    'EndSection Name: A',
    'Text between sections',
    'StartSection Name: B',
    'EndSection Name: B',
    'More text to be ignored',
    ]

In [ ]:
sub_section = Section(
    section_name='Sub-Section',
    start_section=SectionBreak('StartSection', break_offset='Before',
                               name='SubSectionStart'),
    end_section=SectionBreak('EndSection', break_offset='After', 
                             name='SubSectionEnd')
    )
full_section = Section(
    section_name='Top Section',
    end_section=SectionBreak('ignored', break_offset='Before', 
                             name='End Section'),
    processor=sub_section
    )
a = full_section.read(GENERIC_TEST_TEXT, context={'Dummy': 'Blank1'})
print()
print('full_section context')
pprint(full_section.context)
pprint(full_section.scan_status)
print()
print('sub_section context')
pprint(sub_section.context)
pprint(sub_section.scan_status)


full_section context
{'Break': 'End Section',
 'Current Section': 'Top Section',
 'Dummy': 'Blank1',
 'Event': 'ignored',
 'Skipped Lines': [],
 'Status': 'Break Triggered'}
'Break Triggered'

sub_section context
{'Break': 'SubSectionStart',
 'Current Section': 'Sub-Section',
 'Dummy': 'Blank1',
 'Event': 'StartSection',
 'Skipped Lines': ['Text between sections'],
 'Status': 'End of Source'}
'End of Source'


**Context Tests**
<style type="text/css">
.tg th{font-weight:bold;text-align:center;vertical-align:middle}
.tg td{font-weight:normal;text-align:left;vertical-align:top}
.rh td{font-weight:bold;text-align:center;vertical-align:middle}
</style>

<table class="tg">
    <thead>
        <tr>
            <th rowspan="2" text-align="center">Key</th>
            <th colspan="3">Top Section</th>
            <th colspan="3">Sub-Section</th>
        </tr><tr>
            <th>Expected Value</th><th>Actual Value</th><th>Test</th>
            <th>Expected Value</th><th>Actual Value</th><th>Test</th>
        </tr>
    </thead>
        <tr><td class="rh">Current Section</td>
            <td>'Top Section'</td>
            <td>'Top Section'</td>
            <td><img src="../examples/Valid.png" alt="Good"/></td>
            <td>'Sub-Section'</td>
            <td>'Sub-Section'</td>
            <td><img src="../examples/Valid.png" alt="Good"/></td>
        </tr>
    <tr><td class="rh">Dummy</td>
        <td>'Blank1'</td>  
        <td>'Blank1'</td>
        <td><img src="../examples/Valid.png" alt="Good"/></td>
        <td>'Blank1'</td>  
        <td>'Blank1'</td>
        <td><img src="../examples/Valid.png" alt="Good"/></td>
    </tr>
    <tr><td class="rh">Skipped Lines</td>  
        <td>[]</td>
        <td>[]</td>            
        <td><img src="../examples/Valid.png" alt="Good"/></td>
        <td>['Text between sections']</td>
        <td>['Text between sections']</td>            
        <td><img src="../examples/Valid.png" alt="Good"/></td>
    </tr>
    <tr><td class="rh">Status</td>  
        <td>'Break Triggered'</td>
        <td>'Break Triggered'</td>            
        <td><img src="../examples/Valid.png" alt="Good"/></td>
        <td>'End of Source'</td>
        <td>'End of Source'</td>            
        <td><img src="../examples/Valid.png" alt="Good"/></td>
    </tr>
    <tr><td class="rh">Break</td>  
        <td>'End Section'</td>
        <td>'End Section'</td>            
        <td><img src="../examples/Valid.png" alt="Good"/></td>
        <td>'SubSectionStart'</td>
        <td>'SubSectionStart'</td>            
        <td><img src="../examples/Valid.png" alt="Good"/></td>
    </tr>
    <tr><td class="rh">Event</td>  
        <td>'ignored'</td>
        <td>'ignored'</td>            
        <td><img src="../examples/Valid.png" alt="Good"/></td>
        <td>'StartSection'</td>
        <td>'StartSection'</td>            
        <td><img src="../examples/Valid.png" alt="Good"/></td>
    </tr>
</table>

- ***Current Section*** is the name of the section given at the section definition.
    - For _full_section_,  `name='End Section'`
    - For _sub_section_,   `name='Sub-Section'`
- ***Dummy*** is a context element supplied at the time `full_section.read()` was 
  called. Since it is not one of the standard context items it is just along for 
  the ride unless a user defined function explicitly modifies it. 
    - The initially supplied value of ***Dummy*** = _'Blank1'_ is the same for 
      both _full_section_ and _sub_section_.
- ***Skipped Lines*** is the list of unused lines encountered before the beginning 
  of the section.
    - For _full_section_, start_section is not given so ***Skipped Lines*** is 
      an empty list.
    - For _sub_section_, ***Skipped Lines*** is the lines skipped before the 
      most recent sub-section: ['Text between sections']
- ***Status*** is the section state. 
    - _full_section_ has ended because the `end_section` break has triggered, so
      ***Status*** = 'Break Triggered'
    - _sub_section_ receives it's source from _full_section_, but 
      _full_section_ has ended.  _sub_section_ interprets this as and end to 
      it's source, so ***Status*** = 'End of Source'
- ***Break*** is the name of the most recent break triggered while the section is 
   active.
    - The `end_section` break of _full_section_ has triggered.  The name of 
      this SectionBreak was given as 'End Section', so 
      ***Break*** = 'End Section'.
    - The `end_section` break of _sub_section_ never triggered because it's 
      source ended first. The most recent section break for _sub_section_ was 
      the `start_section`, so ***Break*** = 'SubSectionStart'.
- ***Event*** is the Trigger specific value resulting from the last break.  For 
  string based breaks, ***Event*** is the string used to test for the break.
    - The `end_section` break of _full_section_ was triggered by the string 
      'ignored', so ***Event*** = 'ignored'.
    - The `start_section` break of _sub_section_ was triggered by the string 
      'StartSection', so ***Event*** = 'StartSection'.

In [ ]:
class TestSubsectionContext(unittest.TestCase):
    '''Check full_section and sub_section context dictionaries after reading.
    '''
    def setUp(self):
        self.test_text = [
            'Text to be ignored',
            'StartSection A',
            'EndSection A',
            'Text between sections',
            'StartSection B',
            'EndSection B',
            'More text to be ignored',
            ]

    def test_repeat_calls(self):
        '''Verify that repeat calls to read produce the same result.
        '''
        sub_section = Section(
            section_name='SubSection',
            start_section=SectionBreak('StartSection', break_offset='Before',
                                       name='SubSectionStart'),
            end_section=SectionBreak('EndSection', break_offset='After',
                                     name='SubSectionEnd')
            )
        full_section = Section(
            section_name='Top Section',
            end_section=SectionBreak('ignored', break_offset='Before',
                             name='End Section'),
            processor=sub_section
            )
        read_1 = full_section.read(self.test_text, context={'Dummy': 'Blank1'})
        self.assertListEqual(read_1,
                             [['StartSection A', 'EndSection A'],
                              ['StartSection B', 'EndSection B']])
        self.assertDictEqual(full_section.context, {
            'Break': 'End Section',
            'Current Section': 'Top Section',
            'Dummy': 'Blank1',
            'Event': 'ignored',
            'Skipped Lines': [],
            'Status': 'Break Triggered'
            })
        self.assertDictEqual(sub_section.context, {
            'Break': 'SubSectionStart',
            'Current Section': 'SubSection',
            'Dummy': 'Blank1',
            'Event': 'StartSection',
            'Skipped Lines': ['Text between sections'],
            'Status': 'End of Source'
            })

context (Dict[str, Any]): The primary mechanism for the processing
    and aggregation methods to pass contextual information. Break
    point results are is the most commonly used information and are
    automatically added to context.
    When a section boundary is encountered (including sub-sections)
    two items will be added to the context dictionary:
        'Break': (str): The name of the Trigger instance that
            activated the boundary condition.
        'Event' (bool, str, re.match): Information on the
            boundary condition returned by the Trigger instance.
                If Trigger always passes, 'Event' will be True.
                If Trigger matched a string, bool, 'Event' will
                    be the matching string.
                If Trigger matched a regular expression, 'Event'
                    will be the resulting re.match object.
    Additional information can also be stored in context.

    context (ContextType, optional): Break point information and any
        additional information to be passed to and from the
        Section instance.

In [2]:

import sections
?sections.Section.read


Signature:
sections.Section.read(
    self,
    source: 'Source',
    start_search: 'bool' = None,
    do_reset: 'bool' = True,
    initialize: 'bool' = True,
    context: 'ContextType' = None,
) -> 'AggregatedItem'
Docstring:
The primary outward facing section reader function.

Initialize the source and then provide the generator that will step
through all items from source that are in section, applying the section
processing methods and subsection readers to each item.

Step through section_iter If only one sub-section is defined in
self.subsections, yield the aggregate result for that subsection as a
single item from the generator. If multiple sub-sections are defined in
self.subsections, yield a list of the aggregate results for
all of the sub-sections as a single item from the generator.

Arguments:
    source (Source): An iterable where some of the content meets the
        section boundary conditions.
    start_search (bool, optional): Indicates whether to advance through
      